In [27]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import PIL
import cv2  # Added for image preprocessing
from tensorflow import keras
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tqdm import tqdm  # For progress bar
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

def preprocess_image(image_path):
    """Loads and preprocesses an image: converting to grayscale, resizing, normalizing, and applying edge detection."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
    img = cv2.resize(img, (224, 224))  # Resize image

    # Apply Contrast Limited Adaptive Histogram Equalization (CLAHE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img = clahe.apply(img)

    # Apply Gaussian blur to reduce noise
    img = cv2.GaussianBlur(img, (3, 3), 0)

    # Extract edges using Canny edge detection
    edges = cv2.Canny(img, threshold1=100, threshold2=200)

    # Normalize images to [0,1] range
    img = img / 255.0
    edges = edges / 255.0

    # Stack grayscale and edge detection results to create a 2-channel image
    img = np.stack([img, edges], axis=-1)  # Shape: (224, 224, 2)
    
    return img


# Load training data
df_train = pd.read_csv("fungi_train.csv")
df_test = pd.read_csv("fungi_test.csv")

# Encode ClassId numerically
encoder = LabelEncoder()
df_train["ClassId"] = encoder.fit_transform(df_train["ClassId"])

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(df_train["ClassId"]), y=df_train["ClassId"])
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Ensure balanced train-validation split
train_samples_per_class = 800
val_samples_per_class = 200

train_list = []
val_list = []

for class_id in df_train["ClassId"].unique():
    class_subset = df_train[df_train["ClassId"] == class_id]
    train_list.append(class_subset.sample(n=train_samples_per_class, random_state=42))
    val_list.append(class_subset.drop(train_list[-1].index).sample(n=val_samples_per_class, random_state=42))

df_train_balanced = pd.concat(train_list).reset_index(drop=True)
df_val_balanced = pd.concat(val_list).reset_index(drop=True)

# Convert relative paths to absolute paths
df_train_balanced["Path"] = df_train_balanced["Path"].apply(lambda x: os.path.abspath(x))
df_val_balanced["Path"] = df_val_balanced["Path"].apply(lambda x: os.path.abspath(x))

# Image parameters
IMG_SIZE = (224, 224)  # Resize images
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

df_train_balanced["ClassId"] = df_train_balanced["ClassId"].astype(str)
df_val_balanced["ClassId"] = df_val_balanced["ClassId"].astype(str)

train_generator = datagen.flow_from_dataframe(
    dataframe=df_train_balanced,
    directory=None,
    x_col="Path",
    y_col="ClassId",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_generator = datagen.flow_from_dataframe(
    dataframe=df_val_balanced,
    directory=None,
    x_col="Path",
    y_col="ClassId",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

# Load pretrained model
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True  # Unfreeze for fine-tuning

for layer in base_model.layers[:100]:
    layer.trainable = False

# Define model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(224, activation='relu'),
    tf.keras.layers.BatchNormalization(),  # Added for stability
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(np.unique(df_train_balanced["ClassId"])), activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    class_weight=class_weight_dict,
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)]
)
# Save the trained model to a .h5 file
model.save("best_fungi_model.h5")
print("Model saved as best_fungi_model.h5")

# Evaluate model performance
val_preds = np.argmax(model.predict(val_generator), axis=1)
y_true = df_val_balanced["ClassId"].astype(int).values
print(classification_report(y_true, val_preds))
sns.heatmap(confusion_matrix(y_true, val_preds), annot=True, fmt='d')
plt.show()

# Load and preprocess test images
test_images = np.array([preprocess_image(row.Path) for row in tqdm(df_test.itertuples(), total=len(df_test), desc='Processing Test Images')])

# Predict classes
test_preds = np.argmax(model.predict(test_images), axis=1)

# Save predictions
df_submission = pd.DataFrame({"id": df_test["id"], "output": test_preds})
df_submission.to_csv("submission.csv", index=False)
print("Prediction complete. Results saved to submission.csv")


In [29]:
import cv2
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import load_model

# Load training and test data
df_train = pd.read_csv("fungi_train.csv")
df_test = pd.read_csv("fungi_test.csv")

# Convert relative paths to absolute paths
df_train["Path"] = df_train["Path"].apply(lambda x: os.path.abspath(x))
df_test["Path"] = df_test["Path"].apply(lambda x: os.path.abspath(x))

# Encode ClassId numerically
encoder = LabelEncoder()
df_train["ClassId"] = encoder.fit_transform(df_train["ClassId"])

# Initialize ORB detector
orb = cv2.ORB_create(nfeatures=500)

# Store keypoints and descriptors for each class
class_descriptors = {i: [] for i in df_train["ClassId"].unique()}

def extract_orb_features(image_path):
    """Extract ORB keypoints and descriptors from an image."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None, None
    keypoints, descriptors = orb.detectAndCompute(img, None)
    return keypoints, descriptors

# Compute ORB features for training images
for row in tqdm(df_train.itertuples(), total=len(df_train), desc="Extracting ORB Features"):
    _, descriptors = extract_orb_features(row.Path)
    if descriptors is not None:
        class_descriptors[row.ClassId].append(descriptors)

# Function to match descriptors
def match_features(test_desc, train_desc_list):
    """Match ORB descriptors between test and training images."""
    if test_desc is None:
        return 0
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    best_matches = 0
    for train_desc in train_desc_list:
        matches = matcher.match(test_desc, train_desc)
        best_matches = max(best_matches, len(matches))
    return best_matches

# Load trained CNN model as a fallback
#cnn_model = load_model("best_fungi_model.h5")
IMG_SIZE = (224, 224)

def preprocess_image_for_cnn(image_path):
    """Load and preprocess an image for CNN classification."""
    img = cv2.imread(image_path)
    img = cv2.resize(img, IMG_SIZE)
    img = img / 255.0  # Normalize
    return np.expand_dims(img, axis=0)  # Add batch dimension

# Predict test images
predictions = []
for row in tqdm(df_test.itertuples(), total=len(df_test), desc="Matching Test Images"):
    _, test_desc = extract_orb_features(row.Path)
    match_scores = {class_id: match_features(test_desc, class_descriptors[class_id]) for class_id in class_descriptors}
    best_class = max(match_scores, key=match_scores.get)
    
    # If ORB confidence is low, use CNN fallback
   # if match_scores[best_class] < 10:  # Threshold to determine confidence
    #    img = preprocess_image_for_cnn(row.Path)
     #   best_class = np.argmax(cnn_model.predict(img))
    
    predictions.append(best_class)

# Save predictions
df_submission = pd.DataFrame({"id": df_test["id"], "output": predictions})
df_submission.to_csv("submission.csv", index=False)
print("Prediction complete. Results saved to submission.csv")


Matching Test Images: 100%|██████████| 1801/1801 [10:17<00:00,  2.92it/s]

Prediction complete. Results saved to submission.csv


In [1]:
import cv2
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split

# Load training and test data
df_train = pd.read_csv("fungi_train.csv")
df_test = pd.read_csv("fungi_test.csv")

# Convert relative paths to absolute paths
df_train["Path"] = df_train["Path"].apply(lambda x: os.path.abspath(x))
df_test["Path"] = df_test["Path"].apply(lambda x: os.path.abspath(x))

# Encode ClassId numerically
encoder = LabelEncoder()
df_train["ClassId"] = encoder.fit_transform(df_train["ClassId"])

# Initialize ORB detector
orb = cv2.ORB_create(nfeatures=30)

# Store keypoints and descriptors for each class
class_descriptors = {i: [] for i in df_train["ClassId"].unique()}

def extract_orb_features(image_path):
    """Extract ORB keypoints and descriptors from an image."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None, None
    keypoints, descriptors = orb.detectAndCompute(img, None)
    return keypoints, descriptors

# Compute ORB features for training images
for row in tqdm(df_train.itertuples(), total=len(df_train), desc="Extracting ORB Features"):
    _, descriptors = extract_orb_features(row.Path)
    if descriptors is not None:
        class_descriptors[row.ClassId].append(descriptors)

# Function to match descriptors
def match_features(test_desc, train_desc_list):
    """Match ORB descriptors between test and training images."""
    if test_desc is None:
        return 0
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    best_matches = 0
    for train_desc in train_desc_list:
        matches = matcher.match(test_desc, train_desc)
        best_matches = max(best_matches, len(matches))
    return best_matches

# CNN Model Definition
def create_cnn_model(input_shape, num_classes):
    model = keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Prepare data for CNN
IMG_SIZE = (224, 224)
def load_and_preprocess_image(image_path, label):
    img = cv2.imread(image_path)
    img = cv2.resize(img, IMG_SIZE)
    return img, label

# Load and preprocess training data
X_train = []
y_train = []
for row in tqdm(df_train.itertuples(), total=len(df_train), desc="Loading Training Data"):
    img, label = load_and_preprocess_image(row.Path, row.ClassId)
    X_train.append(img)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create and compile the CNN model
input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3)
num_classes = len(df_train["ClassId"].unique())
cnn_model = create_cnn_model(input_shape, num_classes)
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model
cnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Predict test images
predictions = []
for row in tqdm(df_test.itertuples(), total=len(df_test), desc="Matching Test Images"):
    _, test_desc = extract_orb_features(row.Path)
    match_scores = {class_id: match_features(test_desc, class_descriptors[class_id]) for class_id in class_descriptors}
    best_class = max(match_scores, key=match_scores.get)
    
    # If ORB confidence is low, use CNN fallback
    if match_scores[best_class] < 10:  # Threshold to determine confidence
        img, _ = load_and_preprocess_image(row.Path, 0)  # Label is not used here
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        best_class = np.argmax(cnn_model.predict(img))
    
    predictions.append(best_class)

# Save predictions
df_submission = pd.DataFrame({"id": df_test["id"], "output": predictions})
df_submission.to_csv("submission.csv", index=False)
print("Prediction complete. Results saved to submission.csv")

Loading Training Data: 100%|██████████| 5000/5000 [00:02<00:00, 1672.17it/s]


Epoch 1/10


c:\Documents\DATATHON\fungi-classification-shallow-learning\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 30s 231ms/step - accuracy: 0.4518 - loss: 120.9215 - val_accuracy: 0.7110 - val_loss: 0.6039
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 229ms/step - accuracy: 0.5605 - loss: 1.2372 - val_accuracy: 0.1930 - val_loss: 1.6103
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 229ms/step - accuracy: 0.2019 - loss: 1.6044 - val_accuracy: 0.1810 - val_loss: 1.5623
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 231ms/step - accuracy: 0.2280 - loss: 1.8212 - val_accuracy: 0.2190 - val_loss: 1.6076
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 231ms/step - accuracy: 0.2090 - loss: 1.7376 - val_accuracy: 0.1950 - val_loss: 1.6050
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 232ms/step - accuracy: 0.2103 - loss: 1.6087 - val_accuracy: 0.1910 - val_loss: 1.6091
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 232ms/step - accuracy: 0.2113 - loss: 1.6140 - val_accuracy: 0.1920 - val_loss: 1.6089
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 232ms/step - accuracy: 0.2119 - loss: 1.6321 - v

Matching Test Images:   0%|          | 0/1801 [00:00<?, ?it/s]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


Matching Test Images: 100%|█████████▉| 1799/1801 [02:35<00:00, 12.93it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


Matching Test Images: 100%|██████████| 1801/1801 [02:35<00:00, 11.60it/s]

Prediction complete. Results saved to submission.csv


In [4]:
import cv2
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from keras._tf_keras.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Load training and test data
df_train = pd.read_csv("fungi_train.csv")
df_test = pd.read_csv("fungi_test.csv")

# Convert relative paths to absolute paths
df_train["Path"] = df_train["Path"].apply(os.path.abspath)
df_test["Path"] = df_test["Path"].apply(os.path.abspath)

# Encode ClassId numerically
encoder = LabelEncoder()
df_train["ClassId"] = encoder.fit_transform(df_train["ClassId"])

# Initialize ORB detector
orb = cv2.ORB_create(nfeatures=30)

# Store keypoints and descriptors for each class
class_descriptors = {i: [] for i in df_train["ClassId"].unique()}

def extract_orb_features(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    return orb.detectAndCompute(img, None)[1]  # Return only descriptors

# Compute ORB features for training images
for row in tqdm(df_train.itertuples(), total=len(df_train), desc="Extracting ORB Features"):
    descriptors = extract_orb_features(row.Path)
    if descriptors is not None:
        class_descriptors[row.ClassId].append(descriptors)

# Function to match descriptors
def match_features(test_desc, train_desc_list):
    if test_desc is None:
        return 0
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    return max((len(matcher.match(test_desc, train_desc)) for train_desc in train_desc_list), default=0)

# CNN Model Definition
def create_cnn_model(input_shape, num_classes):
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(128, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Prepare data for CNN
IMG_SIZE = (224, 224)

def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    return cv2.resize(img, IMG_SIZE) if img is not None else np.zeros((*IMG_SIZE, 3), dtype=np.uint8)

# Load and preprocess training data efficiently
X_train = np.array([load_and_preprocess_image(row.Path) for row in tqdm(df_train.itertuples(), total=len(df_train), desc="Loading Training Data")])
y_train = df_train["ClassId"].values

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create and compile the CNN model
input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3)
num_classes = len(df_train["ClassId"].unique())
cnn_model = create_cnn_model(input_shape, num_classes)
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model with optimized batch processing
cnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32, verbose=1)

# Predict test images
predictions = []
for row in tqdm(df_test.itertuples(), total=len(df_test), desc="Matching Test Images"):
    test_desc = extract_orb_features(row.Path)
    match_scores = {class_id: match_features(test_desc, class_descriptors[class_id]) for class_id in class_descriptors}
    best_class = max(match_scores, key=match_scores.get)
    
    if match_scores[best_class] < 10:
        img = np.expand_dims(load_and_preprocess_image(row.Path), axis=0)
        best_class = np.argmax(cnn_model.predict(img, verbose=0))
    
    predictions.append(best_class)

# Save predictions
df_submission = pd.DataFrame({"id": df_test["id"], "output": predictions})
df_submission.to_csv("submission.csv", index=False)
print("Prediction complete. Results saved to submission.csv")

Loading Training Data: 100%|██████████| 5000/5000 [00:03<00:00, 1575.46it/s]


Epoch 1/10


c:\Documents\DATATHON\fungi-classification-shallow-learning\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 31s 236ms/step - accuracy: 0.3274 - loss: 123.4690 - val_accuracy: 0.4040 - val_loss: 1.2239
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 233ms/step - accuracy: 0.4656 - loss: 1.1408 - val_accuracy: 0.4350 - val_loss: 1.1024
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 232ms/step - accuracy: 0.5780 - loss: 0.9911 - val_accuracy: 0.1940 - val_loss: 1.9258
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 232ms/step - accuracy: 0.1992 - loss: 1.8038 - val_accuracy: 0.2180 - val_loss: 1.6118
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 232ms/step - accuracy: 0.2542 - loss: 1.7724 - val_accuracy: 0.1980 - val_loss: 1.6484
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 233ms/step - accuracy: 0.2122 - loss: 1.7602 - val_accuracy: 0.4060 - val_loss: 1.5194
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 233ms/step - accuracy: 0.1956 - loss: 1.6377 - val_accuracy: 0.2160 - val_loss: 1.6099
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 232ms/step - accuracy: 0.2097 - loss: 1.6134 - v

Matching Test Images: 100%|██████████| 1801/1801 [02:18<00:00, 12.98it/s]

Prediction complete. Results saved to submission.csv
